In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import xgboost

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor
from sklearn.linear_model import ElasticNet
from sklearn.metrics import mean_squared_error,mean_absolute_percentage_error

from sklearn.preprocessing import MinMaxScaler

## Load data

In [ ]:
main = pd.read_csv('Clean_Main.csv')
domviol = pd.read_csv('Clean_DomesticViolence.csv')
subabu = pd.read_csv('Clean_SubstanceAbuse.csv')
unemp = pd.read_csv('Clean_Unemployment_Data.csv')
foodins = pd.read_csv('extrapolated_food_insecure.csv')
pop = pd.read_csv('Clean_Population.csv')

In [ ]:
regionmap = pd.read_csv('Region_County_mapping.csv')

In [ ]:
main['County'] = main['County'].str.lower()
domviol['County'] = domviol['County'].str.lower()
subabu['County'] = subabu['County'].str.lower()
unemp['County'] = unemp['County'].str.lower()
foodins['County'] = foodins['County'].str.lower()
pop['County'] = pop['County'].str.lower()

domviol['DomesticViolence'] = domviol['DomesticViolence'].str.replace(',','').astype(float)

In [ ]:
main.shape

## Merge

In [ ]:
merged = pd.merge(main,domviol,on=['County','Year'], how='left')

In [ ]:
merged = pd.merge(merged,subabu,on=['County','Year'], how='left')

In [ ]:
merged = pd.merge(merged,unemp,on=['County','Year'])

In [ ]:
merged = pd.merge(merged,foodins,on=['County','Year'])

In [ ]:
merged = pd.merge(merged,pop,on=['County','Year'])

In [ ]:
merged.drop(['Area Name','Region','Percent Overweight','Percent Obese','Percent Healthy Weight'],axis=1,inplace=True)

In [ ]:
merged = merged[~merged['Percent Overweight or Obese'].isna()]

In [ ]:
merged.head()

## Normalize

In [ ]:
merged['DomesticViolence'] = merged['DomesticViolence'].fillna(0) / merged['Population_y']
merged['Admissions'] = merged['Admissions'] / merged['Population_y']
merged['# Uninsured'] = merged['# Uninsured'] / merged['Population_y']
merged['# Uninsured.1'] = merged['# Uninsured.1'] / merged['Population_y']

merged.drop(['Population_y','Population_x'],axis=1, inplace=True)
merged.rename(columns={
    'Percent Overweight or Obese':'OverweightObeseRate',
    'DomesticViolence':'DomesticViolenceRate',
    'Admissions':'SubstanceAbuseAdmissions',
    '% Food Insecure':'FoodInsecurity',
},inplace=True)

In [ ]:
merged['SubstanceAbuseAdmissions'].fillna(0,inplace=True)

In [ ]:
lat = {
    'albany':42.6526,
    'allegany':42.3130,
    'broome':42.0987,
    'cattaraugus':42.2318,
    'cayuga':42.7655,
    'chautauqua':42.2313,
    'chemung':42.1362,
    'chenango':42.4972,
    'clinton':44.7904,
    'columbia':42.3679,
    'cortland':42.5441,
    'delaware':42.2452,
    'dutchess':41.7784,
    'erie':42.9024,
    'essex':44.0107,
    'franklin':44.5926,
    'fulton':43.1119,
    'genesee':42.9838,
    'greene':42.2957,
    'hamilton':43.4764,
    'herkimer':43.1631,
    'jefferson':44.0607,
    'lewis':43.8401,
    'livingston':42.7577,
    'madison':42.9806,
    'monroe':43.2841,
    'montgomery':42.9155,
    'nassau':40.6546,
    'niagara':43.3119,
    'oneida':43.2372,
    'onondaga':43.0268,
    'ontario':42.8510,
    'orange':41.3912,
    'orleans':43.4089,
    'oswego':43.4825,
    'otsego':42.5780,
    'putnam':41.4351,
    'rensselaer':42.6737,
    'rockland':41.1489,
    'st. lawrence':44.4473,
    'saratoga':43.0324,
    'schenectady':42.8493,
    'schoharie':42.6550,
    'schuyler':42.3796,
    'seneca':42.7652,
    'steuben':42.3210,
    'suffolk':40.9849,
    'sullivan':41.6897,
    'tioga':42.1256,
    'tompkins':42.4576,
    'ulster':41.8586,
    'warren':43.6079,
    'washington':43.2519,
    'wayne':43.2020,
    'westchester':41.1220,
    'wyoming':42.6421,
    'yates':42.6431
}

lon = {
    'albany':73.7562,
    'allegany':78.0195,
    'broome':75.9180,
    'cattaraugus':78.7476,
    'cayuga':76.5488,
    'chautauqua':79.5603,
    'chemung':76.7798,
    'chenango':75.6208,
    'clinton':73.6006,
    'columbia':73.5594,
    'cortland':75.9928,
    'delaware':74.8741,
    'dutchess':73.7478,
    'erie':78.8662,
    'essex':73.9508,
    'franklin':74.3388,
    'fulton':74.4995,
    'genesee':78.1564,
    'greene':74.1240,
    'hamilton':74.4057,
    'herkimer':74.8741,
    'jefferson':75.9928,
    'lewis':75.4345,
    'livingston':77.8367,
    'madison':75.8069,
    'monroe':77.7452,
    'montgomery':74.4526,
    'nassau':73.5594,
    'niagara':78.7476,
    'oneida':75.4345,
    'onondaga':76.1784,
    'ontario':77.2865,
    'orange':74.3118,
    'orleans':78.2020,
    'oswego':76.1784,
    'otsego':75.0611,
    'putnam':73.7949,
    'rensselaer':73.5594,
    'rockland':73.9830,
    'st. lawrence':74.9302,
    'saratoga':73.9360,
    'schenectady':73.9830,
    'schoharie':74.4995,
    'schuyler':76.8721,
    'seneca':76.8721,
    'steuben':77.3784,
    'suffolk':72.6151,
    'sullivan':74.7805,
    'tioga':76.3637,
    'tompkins':76.6488,
    'ulster':74.3118,
    'warren':73.7478,
    'washington':73.3709,
    'wayne':77.0104,
    'westchester':73.7949,
    'wyoming':78.2020,
    'yates':77.1485
    
}

In [ ]:
merged['latitude'] = merged.County.map(lat)
merged['longitude'] = merged.County.map(lon)

In [ ]:
merged['state'] = "New York"
merged['country'] = "US"

In [ ]:
merged['longitude']=merged['longitude']*-1

In [ ]:
regionmap = regionmap.iloc[:,:2]
merged_new = pd.merge(merged,regionmap,on='County')

In [ ]:
merged_new.groupby('Region').OverweightObeseRate.mean()

In [ ]:
merged_new.to_csv('merged_data_raw.csv',index=False)

## One Hot Encode

In [ ]:
merged['Sex'] = merged.Sex.map({'FEMALE':0,'MALE':1})
merged.Sex.dtypes

In [ ]:
merged['Grade Level'] = merged['Grade Level'].map({'ELEMENTARY':0,'MIDDLE/HIGH':1})
merged['Grade Level'].dtypes

## Baseline model

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    merged.drop(['County','Year','OverweightObeseRate','# Uninsured','# Uninsured.1','Homicide Rate'],axis=1), merged['OverweightObeseRate'], test_size=0.33, random_state=42)

In [ ]:
scaler = MinMaxScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

### Decision tree

In [ ]:
clf = DecisionTreeRegressor(random_state=0, max_depth=5)
clf.fit(X_train,y_train)

In [ ]:
np.sqrt(mean_squared_error(clf.predict(X_train),y_train))

In [ ]:
mean_absolute_percentage_error(clf.predict(X_train),y_train)

In [ ]:
np.sqrt(mean_squared_error(clf.predict(X_test),y_test))

In [ ]:
mean_absolute_percentage_error(clf.predict(X_test),y_test)

### Random Forest

In [ ]:
clf = RandomForestRegressor(max_samples=7000,
                            bootstrap=True,
                            random_state=0, 
                            max_depth=7, 
                            n_estimators=500, 
                            max_features=3, 
                            min_samples_split=100)
clf.fit(X_train,y_train)

In [ ]:
np.sqrt(mean_squared_error(clf.predict(X_train),y_train))

In [ ]:
mean_absolute_percentage_error(clf.predict(X_train),y_train)

In [ ]:
np.sqrt(mean_squared_error(clf.predict(X_test),y_test))

In [ ]:
mean_absolute_percentage_error(clf.predict(X_test),y_test)

### Extra trees

In [ ]:
clf = ExtraTreesRegressor(random_state=0, max_depth=7, n_estimators=200, max_features=5)
clf.fit(X_train,y_train)

In [ ]:
np.sqrt(mean_squared_error(clf.predict(X_train),y_train))

In [ ]:
mean_absolute_percentage_error(clf.predict(X_train),y_train)

In [ ]:
np.sqrt(mean_squared_error(clf.predict(X_test),y_test))

In [ ]:
mean_absolute_percentage_error(clf.predict(X_test),y_test)

### Sid's fav model

In [ ]:
clf = xgboost.XGBRegressor(n_estimators=100, max_depth=4, eta=0.089, subsample=0.7, colsample_bytree=0.8)
clf.fit(X_train,y_train)

In [ ]:
np.sqrt(mean_squared_error(clf.predict(X_train),y_train))

In [ ]:
mean_absolute_percentage_error(clf.predict(X_train),y_train)

In [ ]:
np.sqrt(mean_squared_error(clf.predict(X_test),y_test))

In [ ]:
mean_absolute_percentage_error(clf.predict(X_test),y_test)

### Elastic Net

In [ ]:
clf = ElasticNet()
clf.fit(X_train,y_train)

In [ ]:
np.sqrt(mean_squared_error(clf.predict(X_train),y_train))

In [ ]:
mean_absolute_percentage_error(clf.predict(X_train),y_train)

In [ ]:
np.sqrt(mean_squared_error(clf.predict(X_test),y_test))

In [ ]:
mean_absolute_percentage_error(clf.predict(X_test),y_test)

## Feature importance

In [ ]:
x=merged.drop(['County','Year','OverweightObeseRate','# Uninsured','# Uninsured.1','Homicide Rate'],axis=1)
y=merged['OverweightObeseRate']

clf = RandomForestRegressor(max_samples=7000,
                            bootstrap=True,
                            random_state=0, 
                            max_depth=7, 
                            n_estimators=500, 
                            max_features=3, 
                            min_samples_split=100)
clf.fit(x,y)

In [ ]:
importances = clf.feature_importances_
feature_names = x.columns
sorted_ids = np.argsort(importances)
importances = importances[sorted_ids]
feature_names = feature_names[sorted_ids]

std = np.std([tree.feature_importances_ for tree in clf.estimators_], axis=0)
std = std[sorted_ids]

In [ ]:
forest_importances = pd.Series(importances, index=feature_names)

fig, ax = plt.subplots(figsize=(15,8))
forest_importances.plot.bar(yerr=std, ax=ax)
ax.set_title("Feature importances using MDI")
ax.set_ylabel("Mean decrease in impurity")
fig.tight_layout()
plt.xticks(fontsize=20)

In [ ]:
clf = xgboost.XGBRegressor(n_estimators=100, max_depth=4, eta=0.089, subsample=0.7, colsample_bytree=0.8)
clf.fit(x,y)

In [ ]:
importances = clf.feature_importances_
feature_names = x.columns
sorted_ids = np.argsort(importances)
importances = importances[sorted_ids]
feature_names = feature_names[sorted_ids]

In [ ]:
forest_importances = pd.Series(importances, index=feature_names)

fig, ax = plt.subplots(figsize=(15,8))
forest_importances.plot.bar(ax=ax)
ax.set_title("Feature importances using MDI")
ax.set_ylabel("Mean decrease in impurity")
fig.tight_layout()

In [ ]:
merged.groupby('County').OverweightObeseRate.mean().sort_values(ascending=False)

In [ ]:
merged['OverweightObeseRate'].corr(merged['FoodInsecurity'])

In [ ]:
merged['OverweightObeseRate'].corr(merged['Unemployment Rate'])